In [21]:
from pypots.imputation import SAITS as dualbranch_model

In [22]:
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed

##  calc_mre , calc_mae

In [23]:

from benchpots.datasets import preprocess_italy_air_quality  # Import the preprocess_italy_air_quality function directly

# Provide the 'n_steps' argument
# Call the function to get the data

In [24]:
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed

set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = benchpots.datasets.preprocess_physionet2012(subset="all", rate=0.1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2024-11-08 14:34:32 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2024-11-08 14:34:32 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-11-08 14:34:32 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-11-08 14:34:32 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-11-08 14:34:32 [INFO]: Loaded successfully!
2024-11-08 14:35:06 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2024-11-08 14:35:06 [INFO]: 68826 values masked out in the val set as ground truth, take 9.98% of the original observed values
2024-11-08 14:35:06 [INFO]: 86353 values masked out in the test set as ground truth, take 9.99% of the original observed v

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


In [25]:
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}
# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
}
## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = np.isnan(physionet2012_dataset['test_X_ori']) ^ np.isnan(physionet2012_dataset['test_X'])
test_X_ori = np.nan_to_num(physionet2012_dataset['test_X_ori'])  # metric functions do not accpet input with NaNs, hence fill NaNs with 0

In [33]:
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam


# initialize the model
model = dualbranch_model(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1, 
    MIT_weight=1,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path="D:\proj\Time series imputation\weight_filel\model",
    model_saving_strategy="best",
)

# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
model.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
model_results = model.predict(dataset_for_testing)
model_imputation = model_results["imputation"]

# calculate mean absolute error on the ground truth (artificially-missing values)
testing_mae = calc_mae(
    model_imputation,
    test_X_ori,
    test_X_indicating_mask,
)
print(f"Testing mean absolute error: {testing_mae:.4f}")

2024-11-08 14:37:03 [INFO]: No given device, using default device: cuda
2024-11-08 14:37:03 [INFO]: Model files will be saved to D:\proj\Time series imputation\weight_filel\model\20241108_T143703
2024-11-08 14:37:03 [INFO]: Tensorboard file will be saved to D:\proj\Time series imputation\weight_filel\model\20241108_T143703\tensorboard
2024-11-08 14:37:03 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182
2024-11-08 14:37:09 [INFO]: Epoch 001 - training loss: 0.7248, validation loss: 6.2435
2024-11-08 14:37:09 [INFO]: Saved the model to D:\proj\Time series imputation\weight_filel\model\20241108_T143703\SAITS_epoch1_loss6.2435014051695665.pypots
2024-11-08 14:37:15 [INFO]: Epoch 002 - training loss: 0.5364, validation loss: 6.2122
2024-11-08 14:37:15 [INFO]: Saved the model to D:\proj\Time series imputation\weight_filel\model\20241108_T143703\SAITS_epoch2_loss6.212190834432841.pypots
2024-11-08 14:37:21 [INFO]: Epoch 003 - training loss:

Testing mean absolute error: 0.2427


In [27]:
testing_mre = calc_mre( # call the function correctly
    model_imputation,
    test_X_ori,
    test_X_indicating_mask,
)
print(f"Testing mre : {testing_mre:.4f}")

Testing mre : 0.3531


In [28]:
print(test_X_indicating_mask)

[[[False False False ... False False False]
  [False False False ...  True False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False Fal

In [29]:
print(model_imputation)

[[[-4.29392219e-01 -3.57200325e-01 -2.79296875e-01 ... -3.46589461e-02
   -1.05640221e+00  3.79583798e-03]
  [ 3.89896221e-02 -2.52709717e-01 -1.74883366e-01 ... -1.20010614e-01
   -1.05640221e+00  1.27250291e-02]
  [-4.46544826e-01 -3.46824586e-01 -2.55950212e-01 ... -2.83666067e-02
   -1.05640221e+00 -7.30885006e-03]
  ...
  [-4.37366366e-01 -3.72983336e-01 -3.68832082e-01 ... -2.64587905e-03
   -1.05640221e+00  3.52167524e-02]
  [-3.22087526e-01 -3.78736973e-01 -3.43933761e-01 ... -2.31846236e-02
   -1.05640221e+00  1.12316404e-02]
  [-4.20116931e-01 -3.44024479e-01 -3.59765947e-01 ... -7.70069333e-03
   -1.05640221e+00  2.32745782e-02]]

 [[-3.26734334e-01 -4.21668768e-01 -3.39395195e-01 ... -5.27377650e-02
   -3.35708523e+00  3.87648772e-03]
  [-4.14426029e-01 -4.55937624e-01 -4.01236892e-01 ... -8.86489451e-02
   -5.08544385e-01 -1.77896582e-02]
  [-3.88349056e-01 -4.69787538e-01 -3.59554559e-01 ... -1.61971122e-01
   -5.00287414e-01 -1.79139785e-02]
  ...
  [-3.87043983e-01 -4.1

In [30]:
print(dir(benchpots.datasets))

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'beijing_multisite_air_quality', 'electricity_load_diagrams', 'electricity_transformer_temperature', 'italy_air_quality', 'pems_traffic', 'physionet_2012', 'physionet_2019', 'preprocess_beijing_air_quality', 'preprocess_electricity_load_diagrams', 'preprocess_ett', 'preprocess_italy_air_quality', 'preprocess_pems_traffic', 'preprocess_physionet2012', 'preprocess_physionet2019', 'preprocess_random_walk', 'preprocess_solar_alabama', 'preprocess_ucr_uea_datasets', 'random_walk', 'solar_alabama', 'ucr_uea_datasets']


In [31]:
italy_air_quality_data = preprocess_italy_air_quality(
    subset="all", rate=0.1, n_steps=12  # Replace 12 with your desired sequence length
)  

2024-11-08 14:36:06 [INFO]: You're using dataset italy_air_quality, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/italy_air_quality
2024-11-08 14:36:06 [INFO]: Dataset italy_air_quality has already been downloaded. Processing directly...
2024-11-08 14:36:06 [INFO]: Dataset italy_air_quality has already been cached. Loading from cache directly...
2024-11-08 14:36:06 [INFO]: Loaded successfully!
2024-11-08 14:36:06 [INFO]: Total sample number: 777
2024-11-08 14:36:06 [INFO]: Training set size: 467 (60.10%)
2024-11-08 14:36:06 [INFO]: Validation set size: 155 (19.95%)
2024-11-08 14:36:06 [INFO]: Test set size: 155 (19.95%)
2024-11-08 14:36:06 [INFO]: Number of steps: 12
2024-11-08 14:36:06 [INFO]: Number of features: 13
2024-11-08 14:36:06 [INFO]: Train set missing rate: 10.03%
2024-11-08 14:36:06 [INFO]: Validating set missing rate: 10.04%
2024-11-08 14:36:06 [INFO]: Test set mi

In [32]:
print(italy_air_quality_data.keys())

dict_keys(['n_steps', 'n_features', 'scaler', 'train_X', 'val_X', 'test_X', 'train_X_ori', 'val_X_ori', 'test_X_ori'])


In [34]:
import numpy as np
# assemble the datasets for training
dataset_for_training = {
    "X": italy_air_quality_data['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": italy_air_quality_data['val_X'],
    "X_ori": italy_air_quality_data['val_X_ori'],
}
# assemble the datasets for test
dataset_for_testing = {
    "X": italy_air_quality_data['test_X'],
}
## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
# Use italy_air_quality_data instead of preprocess_italy_air_quality
test_X_indicating_mask = np.isnan(italy_air_quality_data['test_X_ori']) ^ np.isnan(italy_air_quality_data['test_X'])  
test_X_ori = np.nan_to_num(italy_air_quality_data['test_X_ori'])  # metric functions do not accpet input with NaNs, hence fill NaNs with 0